Retrieved index constituent data from:
https://en.wikipedia.org/wiki/S%26P_GSCI
https://en.wikipedia.org/wiki/Bloomberg_Commodity_Index

In [1]:
import eikon as ek
import numpy as np
import pandas as pd
import json
import datetime as dt
import traceback

In [2]:
def initialize_eikon():
    with open("config.json") as f:
        key = json.load(f)["eikon_key"]

    ek.set_app_key(key)

In [3]:
def load_universe():
    """
    Get the universe of futures contracts to trade

    Returns: a dictionary of Eikon continuous futures tickers grouped by the asset subclass

    """
    universe_df = pd.read_csv("universe.csv")

    return_dict = {}
    for c in universe_df.type.unique():
        return_dict[c] = list(universe_df[universe_df.type == c].eikon_ticker)

    return return_dict

In [7]:
def get_price_data(ticker: str, start_date: dt.date, end_date: dt.date):
    """
    Get historical price data on a single contract

    Args:
        ticker: the ticker of the contract to get
        start_date: the start date of the time series
        end_date: the end date of the time series

    Returns: a data frame containing historical price data on the contract

    """
    data = ek.get_data(
        ticker,
        [
            'TR.CLOSEPRICE.date',
            'TR.CLOSEPRICE',
            'TR.ACCUMULATEDVOLUME'
        ],
        {'SDate': start_date.strftime("%Y-%m-%d"), 'Frq': 'D', 'EDate': end_date.strftime("%Y-%m-%d")}
    )

    data = data[0]

    data = data[data.Date != ""].copy()

    data.Date = data.Date.apply(lambda x: dt.date(int(x[0:4]), int(x[5:7]), int(x[8:10])))

    return data

In [5]:
def get_multiple_price_data(universe, start_date, end_date, continuation: int = 1):
    """
    Get a dictionary with historical prices for all futures in the given universe

    :param universe: a dictionary with all the required futures contracts
    :param start_date: the start date to get data from
    :param end_date: the end date to get data from
    :param continuation: the futures continuation to fetc
    :return: a dictionary with historical prices for each of the given futures
    """
    result_collector = dict()

    for asset_class in universe:
        for ticker in universe[asset_class]:
            try:
                ticker = ticker + "c" + str(continuation)
                price_data = get_price_data(ticker, start_date, end_date)
                result_collector[ticker] = price_data
            except:
                continue

    return result_collector

In [4]:
initialize_eikon()
universe = load_universe()

In [5]:
universe

{'Agriculture': ['SB', 'LSU', 'KC', 'ICF', 'CT', 'W', 'KW', 'C', 'S', 'CC'],
 'Base Metals': ['HG', 'SAF', 'SZN', 'SNI', 'SPB'],
 'Energy': ['CL', 'NG', 'LCO', 'LGO', 'RB', 'HO', 'LHO'],
 'Livestock': ['LC', 'LH', 'FC'],
 'Precious Metals': ['GC', 'SI']}

In [69]:
price_data_c1 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1980, 10, 1),
    end_date=dt.date(2022, 1, 1)
)

In [70]:
len(price_data_c1.keys())

27

In [71]:
pd.concat(price_data_c1.values()).to_csv("price_data_c1.csv", index=False)

In [72]:
price_data_c2 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1991, 10, 1),
    end_date=dt.date(2022, 1, 1),
    continuation=2
)

In [73]:
len(price_data_c2.keys())

27

In [74]:
pd.concat(price_data_c2.values()).to_csv("price_data_c2.csv", index=False)

In [75]:
price_data_c3 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1991, 10, 1),
    end_date=dt.date(2022, 1, 1),
    continuation=3
)

In [76]:
len(price_data_c3.keys())

27

In [77]:
pd.concat(price_data_c3.values()).to_csv("price_data_c3.csv", index=False)

In [78]:
price_data_c4 = get_multiple_price_data(
    universe=universe,
    start_date=dt.date(1991, 10, 1),
    end_date=dt.date(2022, 1, 1),
    continuation=3
)

In [79]:
len(price_data_c4.keys())

27

In [80]:
pd.concat(price_data_c4.values()).to_csv("price_data_c4.csv", index=False)

In [91]:
t1 = pd.read_csv("price_data_c1.csv")
t2 = pd.read_csv("price_data_c2.csv")
t3 = pd.read_csv("price_data_c3.csv")
t4 = pd.read_csv("price_data_c4.csv")

t1.drop(columns=["Unnamed: 0"], inplace=True)
t2.drop(columns=["Unnamed: 0"], inplace=True)
t3.drop(columns=["Unnamed: 0"], inplace=True)
t4.drop(columns=["Unnamed: 0"], inplace=True)

t1["Continuation"] = "c1"
t2["Continuation"] = "c2"
t3["Continuation"] = "c3"
t4["Continuation"] = "c4"

t1["Instrument"] = t1["Instrument"].apply(lambda x: x[:-2])
t2["Instrument"] = t2["Instrument"].apply(lambda x: x[:-2])
t3["Instrument"] = t3["Instrument"].apply(lambda x: x[:-2])
t4["Instrument"] = t4["Instrument"].apply(lambda x: x[:-2])

In [92]:
test_format_1 = pd.concat([t1, t2, t3, t4])
test_format_1

,Instrument,Date,Close Price,Accumulated Volume,Continuation
0,SB,1980-10-01,NaN,275.0,c1
1,SB,1987-08-31,5.250,204.0,c1
2,SB,1988-12-30,10.000,155.0,c1
3,SB,1989-12-29,12.450,132.0,c1
4,SB,1991-02-28,8.950,137.0,c1
...,...,...,...,...,...
172512,SI,2021-12-27,23.100,111.0,c4
172513,SI,2021-12-28,23.100,160.0,c4
172514,SI,2021-12-29,22.860,45.0,c4
172515,SI,2021-12-30,23.075,109.0,c4


In [7]:
t1 = pd.read_csv("price_data_c1.csv")
t2 = pd.read_csv("price_data_c2.csv")
t3 = pd.read_csv("price_data_c3.csv")
t4 = pd.read_csv("price_data_c4.csv")

t1.drop(columns=["Unnamed: 0"], inplace=True)
t2.drop(columns=["Unnamed: 0"], inplace=True)
t3.drop(columns=["Unnamed: 0"], inplace=True)
t4.drop(columns=["Unnamed: 0"], inplace=True)

t1["Instrument"] = t1["Instrument"].apply(lambda x: x[:-2])
t2["Instrument"] = t2["Instrument"].apply(lambda x: x[:-2])
t3["Instrument"] = t3["Instrument"].apply(lambda x: x[:-2])
t4["Instrument"] = t4["Instrument"].apply(lambda x: x[:-2])

t2.rename(
    columns={
        "Accumulated Volume": "C2 - Accumulated Volume",
        "Close Price": "C2 - Close Price"
    },
    inplace=True
)

t3.rename(
    columns={
        "Accumulated Volume": "C3 - Accumulated Volume",
        "Close Price": "C3 - Close Price"
    },
    inplace=True
)

t4.rename(
    columns={
        "Accumulated Volume": "C4 - Accumulated Volume",
        "Close Price": "C4 - Close Price"
    },
    inplace=True
)

In [8]:
test_format_2 = t1.merge(t2, how="left").merge(t3, how="left").merge(t4, how="left")
test_format_2[~pd.isnull(test_format_2["C2 - Close Price"])]

,Instrument,Date,Close Price,Accumulated Volume,C2 - Close Price,C2 - Accumulated Volume,C3 - Close Price,C3 - Accumulated Volume,C4 - Close Price,C4 - Accumulated Volume
18,SB,1991-10-01,8.880,15.0,8.770,2007.0,8.730,549.0,8.730,549.0
19,SB,1991-10-02,8.850,152.0,8.740,1317.0,8.720,430.0,8.720,430.0
20,SB,1991-10-03,8.550,82.0,8.480,3825.0,8.480,865.0,8.480,865.0
21,SB,1991-10-04,8.660,52.0,8.590,915.0,8.550,74.0,8.550,74.0
22,SB,1991-10-07,8.750,27.0,8.650,1109.0,8.640,110.0,8.640,110.0
...,...,...,...,...,...,...,...,...,...,...
184232,SI,2021-12-27,22.680,1.0,23.060,108793.0,23.100,111.0,23.100,111.0
184233,SI,2021-12-28,23.150,211.0,23.015,73451.0,23.100,160.0,23.100,160.0
184234,SI,2021-12-29,22.635,172.0,22.825,98976.0,22.860,45.0,22.860,45.0
184235,SI,2021-12-30,23.070,344.0,23.100,75359.0,23.075,109.0,23.075,109.0


# Stock & Bond Indices

In [19]:
bond_index = get_price_data(
    ticker=".SBBIG",
    start_date=dt.date(1980, 10, 1),
    end_date=dt.date(2022, 1, 1)
)

In [20]:
bond_index

,Instrument,Date,Close Price,Accumulated Volume
0,.SBBIG,1980-10-01,<NA>,<NA>
1,.SBBIG,1995-01-03,466.3319,<NA>
2,.SBBIG,1995-01-04,467.9437,<NA>
3,.SBBIG,1995-01-05,467.1224,<NA>
4,.SBBIG,1995-01-06,467.4298,<NA>
...,...,...,...,...
6910,.SBBIG,2021-12-27,1939.9205,<NA>
6911,.SBBIG,2021-12-28,1940.458,<NA>
6912,.SBBIG,2021-12-29,1935.1257,<NA>
6913,.SBBIG,2021-12-30,1938.2683,<NA>


In [21]:
sp500_tr_index = get_price_data(
    ticker=".SPXTR",
    start_date=dt.date(1980, 10, 1),
    end_date=dt.date(2022, 1, 1)
)

In [22]:
sp500_tr_index

,Instrument,Date,Close Price,Accumulated Volume
0,.SPXTR,1980-10-01,<NA>,<NA>
1,.SPXTR,1988-01-05,258.77,0
2,.SPXTR,1988-01-06,259.03,0
3,.SPXTR,1988-01-07,261.211,0
4,.SPXTR,1988-01-08,243.545,0
...,...,...,...,...
8568,.SPXTR,2021-12-27,10037.6683,<NA>
8569,.SPXTR,2021-12-28,10027.5329,<NA>
8570,.SPXTR,2021-12-29,10041.611,<NA>
8571,.SPXTR,2021-12-30,10012.5135,<NA>


In [23]:
russell_2000_tr_index = get_price_data(
    ticker=".RUTTR",
    start_date=dt.date(1980, 10, 1),
    end_date=dt.date(2022, 1, 1)
)

In [24]:
russell_2000_tr_index

,Instrument,Date,Close Price,Accumulated Volume
0,.RUTTR,1980-10-01,<NA>,<NA>
1,.RUTTR,1995-06-02,989.48,<NA>
2,.RUTTR,1995-06-05,996.5,<NA>
3,.RUTTR,1995-06-06,998.22,<NA>
4,.RUTTR,1995-06-07,998.79,<NA>
...,...,...,...,...
6695,.RUTTR,2021-12-27,11701.1068,<NA>
6696,.RUTTR,2021-12-28,11624.1488,<NA>
6697,.RUTTR,2021-12-29,11638.7674,<NA>
6698,.RUTTR,2021-12-30,11639.858,<NA>


In [26]:
bond_index.rename(
    columns={"Close Price": "USBIG"},
    inplace=True
)
bond_index.drop(
    columns=["Accumulated Volume", "Instrument"],
    inplace=True
)

sp500_tr_index.rename(
    columns={"Close Price": "SP500"},
    inplace=True
)
sp500_tr_index.drop(
    columns=["Accumulated Volume", "Instrument"],
    inplace=True
)

russell_2000_tr_index.rename(
    columns={"Close Price": "RUS2000"},
    inplace=True
)
russell_2000_tr_index.drop(
    columns=["Accumulated Volume", "Instrument"],
    inplace=True
)

In [37]:
market_indices = bond_index.merge(sp500_tr_index, how="outer").merge(russell_2000_tr_index, how="outer").dropna()

In [38]:
market_indices.head(5)

,Date,USBIG,SP500,RUS2000
109,1995-06-02,521.2857,675.398,989.48
110,1995-06-05,521.6984,679.3172,996.5
111,1995-06-06,521.2241,679.3641,998.22
112,1995-06-07,519.2062,676.2942,998.79
113,1995-06-08,517.5915,675.3885,1002.14


In [39]:
market_indices["USBIG_Return"] = market_indices["USBIG"] / market_indices["USBIG"].shift(1) - 1
market_indices["SP500_Return"] = market_indices["SP500"] / market_indices["SP500"].shift(1) - 1
market_indices["RUS2000_Return"] = market_indices["RUS2000"] / market_indices["RUS2000"].shift(1) - 1

In [40]:
market_indices

,Date,USBIG,SP500,RUS2000,USBIG_Return,SP500_Return,RUS2000_Return
109,1995-06-02,521.2857,675.398,989.48,<NA>,<NA>,<NA>
110,1995-06-05,521.6984,679.3172,996.5,0.000792,0.005803,0.007095
111,1995-06-06,521.2241,679.3641,998.22,-0.000909,0.000069,0.001726
112,1995-06-07,519.2062,676.2942,998.79,-0.003871,-0.004519,0.000571
113,1995-06-08,517.5915,675.3885,1002.14,-0.00311,-0.001339,0.003354
...,...,...,...,...,...,...,...
6910,2021-12-27,1939.9205,10037.6683,11701.1068,0.000617,0.013875,0.008893
6911,2021-12-28,1940.458,10027.5329,11624.1488,0.000277,-0.00101,-0.006577
6912,2021-12-29,1935.1257,10041.611,11638.7674,-0.002748,0.001404,0.001258
6913,2021-12-30,1938.2683,10012.5135,11639.858,0.001624,-0.002898,0.000094


In [41]:
market_indices.to_csv("market_indices.csv", index=False)